In [88]:
# ! conda create -y -n milvus python=3.10

! pip install pymilvus==2.4.0
! pip install langchain
! pip install ibm-generative-ai
! pip install unstructured
! pip install huggingface-hub
! pip install sentence-transformers
! pip install ibm_watsonx_ai


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [89]:
import os
import requests
import json

from dotenv import load_dotenv

from langchain_community.document_loaders import UnstructuredURLLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Milvus

from langchain.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler

# BAM
from genai.extensions.langchain import LangChainInterface
from genai.credentials import Credentials
from genai import Client

from genai.schema import TextGenerationParameters

from ibm_watsonx_ai.foundation_models import Embeddings

load_dotenv() 

False

In [96]:
token = ""
project_id = ""

In [91]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
text_splitter.split_text("gogo")

['gogo']

In [93]:
embedding_model = Embeddings(
            model_id="ibm/slate-125m-english-rtrvr",
            credentials={"api_key": token, "url": "https://us-south.ml.cloud.ibm.com"},
            project_id=project_id,
            params={
                "truncate_input_tokens": 512
            }
        )
embedding_model

In [94]:
# INITIALIZE A MILVUS COLLECTION WITH EMBEDDINGS FROM A SET OF DOCUMENTS

urls = [
    "https://en.wikipedia.org/wiki/Elon_Musk",
]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [95]:

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

vector_db = Milvus.from_documents(
    docs,
    embedding_model,
    drop_old=True,  # Drop the old Milvus collection if it exists
    connection_args={"host":"823135eb-c10e-4ae5-9d93-62b881faa87b.cpltrlid0d8r2cjtm4e0.lakehouse.appdomain.cloud", 
                     "port":"31660", "secure":True,
                     "user":"ibmlhapikey", "password": token},
)

Created a chunk of size 2116, which is longer than the specified 2000
Created a chunk of size 10275, which is longer than the specified 2000


In [12]:
# TEST COLLECTION INITIALISATION WITH A SIMPLE SIMILARITY SEARCH

query = "What is the net worth of Elon Musk today?"
docs = vector_db.similarity_search(query)

docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Elon_Musk', 'pk': 451797497425623530}, page_content='Elon Musk FRS Musk in 2023 Born Elon Reeve Musk ( 1971-06-28 ) June 28, 1971 (age\xa053) Pretoria , Transvaal , South Africa Citizenship South Africa Canada United States (from 2002) Education University of Pennsylvania ( BA , BS ) Title Founder, CEO, and chief engineer of SpaceX CEO and product architect of Tesla, Inc. Owner, CTO and Executive Chairman of X (formerly Twitter) President of the Musk Foundation Founder of The Boring Company , X Corp. , and xAI Co-founder of Neuralink , OpenAI , Zip2 , and X.com (part of PayPal ) Spouses Justine Wilson \u200b \u200b ( m. 2000; div. 2008) \u200b Talulah Riley \u200b \u200b ( m. 2010; div. 2012) \u200b \u200b \u200b ( m. 2013; div. 2016) \u200b Partners Grimes (2018–2021) [ 1 ] Children 12 [ 2 ] Parents Errol Musk Maye Musk Relatives Kimbal Musk (brother) Tosca Musk (sister) Lyndon Rive (cousin) Awards Full list Signature\n\nThi

In [83]:
# SET LANGCHAIN INTERFACE FOR GEN AI

creds = Credentials(token,
                    api_endpoint="https://us-south.ml.cloud.ibm.com") # credentials object to access the LLM service


params = TextGenerationParameters(decoding_method="sample",temperature=0.7, top_p=1, top_k=50, min_new_tokens=0, max_new_tokens=200
                        )

model = LangChainInterface(model_id= 'ibm/granite-13b-chat-v2', parameters=params, client=Client(credentials=creds))

prompt_template = (
            """[INST]
            <<SYS>>
                here are your instructions:
            1. You will be given a context that should be used to reply to user questions. The context will begin with \"BEGIN\" and will end with \"END\".
            2. You should generate the next response using informations available in the context.
            3. If you can't find an answer, say \"I don't know\".
            4. Do not use any other knowledge.
            5. Specify the sentences in context that justify your answer.
                BEGIN {context} END
            <</SYS>>
            <<HUMAN>>
              {question}
            <</HUMAN>>
            [/INST]
            """
        )

customprompt = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )

In [84]:
model

LangChainInterface(client=<genai.client.Client object at 0x17ba2cc50>, model_id='ibm/granite-13b-chat-v2', parameters=TextGenerationParameters(beam_width=None, decoding_method='sample', include_stop_sequence=None, length_penalty=None, max_new_tokens=200, min_new_tokens=0, random_seed=None, repetition_penalty=None, return_options=None, stop_sequences=None, temperature=0.7, time_limit=None, top_k=50, top_p=1.0, truncate_input_tokens=None, typical_p=None))

In [100]:
def append_docs(docs):
    context = ''
    for doc in docs:
        context += doc.page_content
    return context

def get_wkc_token_on_cloud(apikey):
    body = {"grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": apikey}

    token_request = requests.post('https://iam.cloud.ibm.com/identity/token',
                                  data=body, headers={'content-type': 'application/x-www-form-urlencoded'})
    try:
        token_request.raise_for_status()
    except requests.exceptions.HTTPError as e:
        raise e  # return code not equals to 200

    return json.loads(token_request.text)["access_token"]


'eyJraWQiOiIyMDI0MDgwMzA4NDEiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC01MDhHOFI3MjMyIiwiaWQiOiJJQk1pZC01MDhHOFI3MjMyIiwicmVhbG1pZCI6IklCTWlkIiwianRpIjoiYWM0Njg0NTctZjcyOC00Y2Q0LWIwNzQtNTdhZTFkYmIwYzU3IiwiaWRlbnRpZmllciI6IjUwOEc4UjcyMzIiLCJnaXZlbl9uYW1lIjoiRmxhdmlhIEphbmluZSIsImZhbWlseV9uYW1lIjoiUm9zYW50ZSBCZW8iLCJuYW1lIjoiRmxhdmlhIEphbmluZSBSb3NhbnRlIEJlbyIsImVtYWlsIjoiRmxhdmlhLkJlb0BpYm0uY29tIiwic3ViIjoiRmxhdmlhLkJlb0BpYm0uY29tIiwiYXV0aG4iOnsic3ViIjoiRmxhdmlhLkJlb0BpYm0uY29tIiwiaWFtX2lkIjoiSUJNaWQtNTA4RzhSNzIzMiIsIm5hbWUiOiJGbGF2aWEgSmFuaW5lIFJvc2FudGUgQmVvIiwiZ2l2ZW5fbmFtZSI6IkZsYXZpYSBKYW5pbmUiLCJmYW1pbHlfbmFtZSI6IlJvc2FudGUgQmVvIiwiZW1haWwiOiJGbGF2aWEuQmVvQGlibS5jb20ifSwiYWNjb3VudCI6eyJ2YWxpZCI6dHJ1ZSwiYnNzIjoiNTk1MDZjNjkzOTVmNGIzZjk3OWY1OTRiZDlkYzM4NWEiLCJpbXNfdXNlcl9pZCI6IjEwOTQyMTIxIiwiZnJvemVuIjp0cnVlLCJpbXMiOiIxOTYwMjY2In0sImlhdCI6MTcyMzk0MjIwMCwiZXhwIjoxNzIzOTQ1ODAwLCJpc3MiOiJodHRwczovL2lhbS5jbG91ZC5pYm0uY29tL2lkZW50aXR5IiwiZ3JhbnRfdHlwZSI6InVybjppYm06cGFyYW1zOm9hdXRoOmdyYW

In [103]:
def qa(question):
    docs = vector_db.similarity_search(query)

    print(str(len(docs))+" nearest embeddings found;")
    
    context = append_docs(docs=docs)
    prompt = customprompt.format(context=context, question=question)

    barear_token = get_wkc_token_on_cloud(token)

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": {
            "decoding_method": "greedy",
            "max_new_tokens": 900,
            "repetition_penalty": 1.05
        },
        "model_id": "ibm/granite-13b-chat-v2",
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(barear_token)
    }

    response = requests.post(
        url,
        headers=headers,
        json=body
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))
    
    data = response.json()
    
    return data['results'][0]['generated_text']

In [104]:
qa("What is the net worth of Elon Musk?")

4 nearest embeddings found;


'The net worth of Elon Musk is estimated to be 241 billion dollars as of August 2024.'

In [105]:
qa("What was the college Elon Musk attended?")

4 nearest embeddings found;


"Musk attended the University of Pennsylvania for his undergraduate studies, where he received bachelor's degrees in economics and physics."